<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<p><p><p><p>
<b>Introduction to Rubin Image Services</b> <br>
Contact author: <i>Leanne Guy</i> <br>
Last verified to run: <i>2023-01-13</i> <br>
LSST Science Piplines version: Weekly <i>2022_24</i> <br>
Container Size: <i>medium</i> <br>
Targeted learning level: <i>intermediate</i> <br>

In [1]:
# %load_ext pycodestyle_magic
# %flake8_on
# import logging
# logging.getLogger("flake8").setLevel(logging.FATAL)

**Skills:** Learn how to use Rubin Images Services to query and retrieve images. Learm how to  and image cutouts.

**LSST Image Data Products:** Single-epoch Images, Deep Coadds, 

**Packages:** lsst.rsp.get_tap_service, lsst.rsp.retrieve_query, pyvo

**Credit:** This tutorial was developed for DP0.2 by Leanne Guy.

**Get Support:**
Find DP0-related documentation and resources at <a href="https://dp0-1.lsst.io">dp0-1.lsst.io</a>. Questions are welcome as new topics in the <a href="https://community.lsst.org/c/support/dp0">Support - Data Preview 0 Category</a> of the Rubin Community Forum. Rubin staff will respond to all questions posted there.

## 1. Introduction

Image Services is the term used to describe the collection of Rubin services used for the discovery, description, access, and retrieval of LSST image data products. The term Image Data Products includes   Rubin Observatory has adopted 'VO First' approach for services, meaing that wherever possible, IVOA-standard interfaces will be adopted. 
The Virtual Observatory (VO) is the vision that astronomical datasets and services should work together as a whole. The [International Virtual Observatory Alliance (IVOA)](ivoa.net) is an organisation that debates and agrees the technical standards that are needed to make the VO possible.

Tutorial 02, Catalog Queries with TAP, introduced the first Rubin IVOA-compliant service to be deployed as part of DP0.1 for accessing table data and metadata. The Rubin TAP service returns structured information about LSST data products.  In this tutorial we will introduce two new IVOA-compliant services that have been deployed as part of DP02 for accessing images and creating image cutouts. 

ObsCore

DataLink 

SODA

----
Planned Image services for Rubin will provide the following IVOA service endpoints. 

* Image metadata query: ObsTAP, from ObsCore 1.1 or later (as an initial priority), with SIAv2 (also ObsCore-based and serving equivalent data) later
* Catalog query: TAP 1.1 or later
* Image cutout service: SODA 1.0 or later
* DataLink links service: DataLink 1.0 or later (very likely to be revised in the near future)

DP0.2 deploys early versions of some of these services. In this tutorial we will look at the Image Metadata Query Service and the Image Cutout Service.

Goals of this tutorial

* Intro to ObsCore in the documentatation - not to put in here but to break down
* Explore the ivoa.ObsCore table and understand its use
https://data-int.lsst.cloud/api/cutout/sync?id=8a953c0321bd4878bfa694dbf628ea81&circle=53.13925%20-34.0215%200.0105
* Intro to the SODA service for image cutouts



When working with astronomical images it is often advantageous to work with \"cutouts\". A cutout is a sub-image of an original image centered on a given coordinate and of a given size. This tutorial demonstrates how to use the Rubin Image Cutout Service to query and retrieve image cutouts matching a region of interest specified by the user for DP0.2. 

### 1.1 Package Imports

In [2]:
# Import general python packages
import numpy as np
import re
import pandas
from pandas.testing import assert_frame_equal
import uuid
import requests
import warnings

# Import the Rubin TAP service utilities
from lsst.rsp import get_tap_service, retrieve_query

# Science Pipelines imports
from lsst.daf.butler import Butler, DatasetType, CollectionType
import lsst.geom as geom
import lsst.resources
import lsst.geom as geom
import lsst.afw.image as afwImage
from lsst.afw.image import Image, ImageF
from lsst.afw.image.exposure import Exposure, ExposureF
import lsst.afw.display as afwDisplay

# Plotting with MPL
import matplotlib.pyplot as plt

# Import the Rubin TAP service utilities
from lsst.rsp import get_tap_service
from lsst.rsp import get_tap_service, retrieve_query
from lsst.rsp.utils import get_access_token

# PyVO packages
import pyvo
from pyvo.dal.adhoc import DatalinkResults, SodaQuery

# Astropy
from astropy import units as u
from astropy.units import UnitsWarning
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.time import Time
from astropy.utils.data import download_file
from astropy.visualization import  ZScaleInterval, AsinhStretch
from astropy.wcs import WCS       
from astropy.visualization import simple_norm, imshow_norm
from astropy.visualization import ImageNormalize,  ZScaleInterval
from astropy.visualization.stretch import SinhStretch, LinearStretch, SqrtStretch

# Holoviz for interactive visualization
import bokeh
from bokeh.io import output_file, output_notebook, show
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, CDSView, GroupFilter, HoverTool
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
import holoviews as hv
from holoviews import streams, opts
from holoviews.operation.datashader import rasterize

Check the version of the LSST Science Pipelines

In [3]:
# This should match the verified version listed at the start of the notebook
! echo ${IMAGE_DESCRIPTION}
! eups list lsst_distrib

Recommended (Weekly 2023_07)
   gdf42428520+759eb90f97 	current w_2023_07 setup


In [4]:
# Set up some plotting defaults
params = {'axes.labelsize': 18,
          'font.size': 18,
          'legend.fontsize': 12,
          'xtick.major.width': 2,
          'xtick.minor.width': 1,
          'xtick.major.size': 10,
          'xtick.minor.size': 4,
          'xtick.direction': 'in',
          'xtick.top': True,
          'lines.linewidth': 2,
          'axes.linewidth': 2,
          'axes.labelweight': 2,
          'axes.titleweight': 2,
          'ytick.major.width': 2,
          'ytick.minor.width': 1,
          'ytick.major.size': 10,
          'ytick.minor.size': 4,
          'ytick.direction': 'in',
          'ytick.right': True,
          'figure.figsize': [6, 6],
          'figure.facecolor': 'White'
          }

plt.rcParams.update(params)

In [5]:
def plotImage(exposure: ExposureF, img_opt: dict = None):
    """Plot and image using matplotlib
   
   Parameters
    ----------
    image : `Exposure`
        the image to plot
        
    opts : ``
   
   Returns
    -------
    title : `str` (only if result is not `None`)
        Plot title from string
    """
    
    fig, ax = plt.subplots()
    display = afwDisplay.Display(frame=fig)
    display.scale('asinh', 'zscale')
    display.mtv(exposure.image)
    plt.show()


### 1.2 Define Functions and Parameters

Set a few parameters related to plotting and display.

In [6]:
# Set afw display backend to matplotlib
afwDisplay.setDefaultBackend('matplotlib')

# Set the maximum number of rows to display from pandas
pandas.set_option('display.max_rows', 20)

In [7]:
butler = Butler('dp02', collections='2.2i/runs/DP0.2')
registry = butler.registry

## 2. Data Preparation

Let's define some known objects of interest to work with. 

In [8]:
# Define a spatial point
# SN1a From notebook 7a
diaObjectId = 1250953961339360185


In [9]:
%%time

# Define a spatial point
spherePoint = lsst.geom.SpherePoint(55.7467*geom.degrees, -32.2862*geom.degrees)

#55.75067151347799 -32.27781914678513

# Find the tract and patch that contain this point
skymap = butler.get('skyMap')
tract = skymap.findTract(spherePoint)
patch = tract.findPatch(spherePoint)

# Now create a dataId from the tract, patch and filter information
dataId = {'band':'i', 'tract':tract.tract_id, 'patch': patch.getSequentialIndex()}

# And use the dataid to get the coadd
deepCoadd = butler.get('deepCoadd_calexp', dataId=dataId)

assert type(deepCoadd) == lsst.afw.image.exposure.ExposureF
f"Tract: {tract.tract_id}, Patch: {patch.getSequentialIndex()}"

CPU times: user 3.24 s, sys: 4.06 s, total: 7.3 s
Wall time: 4.92 s


'Tract: 4431, Patch: 17'

In [10]:
# Set up a DP0. 2Butler 
butler = Butler('dp02', collections='2.2i/runs/DP0.2')
registry = butler.registry

In [11]:
## 

## 3 Image Metadata Query 

Understanding the ivoa ObsCore table and its use .....

ObsTAP, from ObsCore 1.1 or later (as an initial priority), with SIAv2 (also ObsCore-based and serving equivalent data) later

As of DP0.2 there is a new schema (table collection) called \"ivoa\", which contains a table called ivoa.ObsCore. 
The IVOA-defined obscore table contains generic metadata for datasets held at the IDF. 
The table is accessible via ADQL queries via a TAP endpoint. The mechanism for locating images from obsevations is to make a TAP query against the ObsCore schema.

<br>
Here are some definitions to help you understand the contents of the ivoa schema. 

* `VO` - Vitrual Observatory - the vision that astronomical datasets and other resources should work as a seamless whole. Rubin is one of many projects and data centres worldwide who are are working towards this goal. Rubin has adopted a \"VO first strategy\".
* `IVOA` - International Virtual Observatory Alliance, ivoa.net. An organization that debates and agrees the technical standards that are needed to make the VO possible
* `ObsTAP` - An IVOA standare
* `ObsCore` - An ..
* `VOTable` - https://www.ivoa.net/documents/VOTable/ -- VOTable format is an XML standard for the interchange of data represented as a set of tables.
* `DataLink` - An IVOA standard to link from metadata about a dataset to the dataset itself, as well as other related data and services that can operate on that data

### 3.1 The ObsCore data model table

let's look at how to query the ObsCore table. We will start by looking at the schemas at the US DAC. In addition to the DP0.1 and DP0.2 and TAP scheam that were present in DP0.1, you will now see a schema called \"ivoa\".

First, using the coordinates defined above, we define a circle with a radius that emcompasses the galaxy cluster.

In [12]:
# Need the UUID of the image for the cutout service 
datasetRef = registry.findDataset('deepCoadd_calexp', dataId)
deepCoadd_uuid = datasetRef.id
assert isinstance(deepCoadd_uuid, uuid.UUID)
f"UUID for deep coadd: {deepCoadd_uuid}"

'UUID for deep coadd: 20d28216-534a-4102-b8a7-1c7f32a9b78c'

In [13]:
service = get_tap_service()

In [14]:
query = "SELECT * FROM tap_schema.schemas"
results = service.search(query)
results = service.search(query).to_table()
results

description,schema_index,schema_name,utype
str512,int32,str64,str512
Data Preview 0.1 includes five tables based on the DESC's Data Challenge 2 simulation of 300 square degrees of the wide-fast-deep LSST survey region after 5 years. All tables contain objects detected in coadded images.,2,dp01_dc2_catalogs,
"Data Preview 0.2 contains the image and catalog products of the Rubin Science Pipelines v23 processing of the DESC Data Challenge 2 simulation, which covered 300 square degrees of the wide-fast-deep LSST survey region over 5 years.",0,dp02_dc2_catalogs,
ObsCore v1.1 attributes in ObsTAP realization,1,ivoa,
A TAP-standard-mandated schema to describe tablesets in a TAP 1.1 service,100000,tap_schema,
UWS Metadata,120000,uws,


Lets look at the tables in the ivoa schema 

In [15]:
query = "SELECT * FROM tap_schema.tables where schema_name like 'ivoa' order by table_index ASC"
result = service.search(query).to_table()
result

description,schema_name,table_index,table_name,table_type,utype
str512,str512,int32,str64,str8,str512
Observation metadata in the ObsTAP relational realization of the IVOA ObsCore data model,ivoa,0,ivoa.ObsCore,table,


The ivoa schema contains 1 table called \"ivoa.ObsCore\". This table holds the observation metadata for all images in the DP0.2 dataset in an ObsTAP realization of the IVOA ObsCore data model

In [16]:
query = "SELECT COUNT(*) from ivoa.ObsCore"
result = service.search(query).to_table()
result

COUNT
int64
8475974


The ivoa.ObsCore contains 8475974 entries for the DP0.2 dataset. Lets's take a quick look at the contents of the ObsCore table. 

There is currenty ony one data product type supported -- \"image\". All entries in the ObsCore table are for images

In [17]:
query = """ SELECT dataproduct_type, COUNT(*) FROM ivoa.ObsCore GROUP BY dataproduct_type
"""
result = service.search(query).to_table()
result

dataproduct_type,COUNT
str128,int64
image,8475974


In [18]:
query = """ SELECT dataproduct_subtype, COUNT(*) FROM ivoa.ObsCore GROUP BY dataproduct_subtype
"""
result = service.search(query).to_table()
result

dataproduct_subtype,COUNT
str64,int64
lsst.calexp,2805017
lsst.deepCoadd_calexp,46158
lsst.goodSeeingCoadd,46158
lsst.goodSeeingDiff_differenceExp,2707834
lsst.raw,2870807


In [19]:
query = """ SELECT calib_level, COUNT(*) 
FROM ivoa.ObsCore 
GROUP BY calib_level
"""
result = service.search(query).to_table()
result

calib_level,COUNT
int32,int64
1,2870807
2,2805017
3,2800150


Looking at the relationship between calib_level and dataproduct_subtype, we see that calib_level of coresponds to lsst.raw data product subtype. 2 to lsst.calexp, 3 to coadds and the difference image resulting from DIA. calib_level 4 contains no images. Change teh number in "where calib_level=1" to 1,2,3,4 and see the results
TODO - make this all 1 SQL command 

In [20]:
query = """ SELECT dataproduct_subtype, COUNT(*) 
FROM ivoa.ObsCore 
where calib_level = '1'
GROUP BY dataproduct_subtype
"""
result = service.search(query).to_table()
result

dataproduct_subtype,COUNT
str64,int64
lsst.raw,2870807


We can use the ObsCore table to select all images in a region as follows. Let's use the point defined previoust with a search radius of 10". The 's_region' column defines the sky region covered by a data product, expressed in the ICRS frame.
TODO - fill in the coordinates from the SpherePoint not copy and paste. We see that there are 6 deepCoadd_calexp images, one for each filter. 

In [21]:
# Select all images in a region
query = """SELECT * FROM ivoa.ObsCore 
WHERE dataproduct_type = 'image'
AND obs_collection = 'LSST.DP02' 
AND dataproduct_subtype = 'lsst.deepCoadd_calexp'
AND CONTAINS(POINT('ICRS', 55.74673760481304, -32.286155241413624), s_region)=1 
"""
results = service.search(query).to_table()
results

access_format,access_url,calib_level,dataproduct_subtype,dataproduct_type,em_max,em_min,em_res_power,em_xel,facility_name,instrument_name,lsst_band,lsst_detector,lsst_filter,lsst_patch,lsst_tract,lsst_visit,o_ucd,obs_collection,obs_id,obs_publisher_did,pol_xel,s_dec,s_fov,s_ra,s_region,s_resolution,s_xel1,s_xel2,t_exptime,t_max,t_min,t_resolution,t_xel,target_name
,,,,,m,m,,,,,,,,,,,,,,,,deg,deg,deg,,arcsec,,,s,d,d,s,,
str128,object,int32,str64,str128,float64,float64,float64,int64,str128,str128,str10,int64,str10,int64,int64,int64,str32,str128,str128,str256,int64,float64,float64,float64,str512,float64,int64,int64,float64,float64,float64,float64,int64,str32
application/x-votable+xml;content=datalink,https://data.lsst.cloud/api/datalink/links?ID=butler%3A//dp02/14e949ec-59a0-4e9a-99c3-dbc74e904fe8,3,lsst.deepCoadd_calexp,image,1.06e-06,9.7e-07,--,--,Rubin-LSST,,y,--,,17,4431,--,phot.count,LSST.DP02,DC2-4431-17,,--,-32.205663176664345,0.3299793994195083,55.65214108466622,POLYGON ICRS 55.514085 -32.322253 55.790197 -32.322253 55.789845 -32.088924 55.514437 -32.088924,--,--,--,--,--,--,--,--,
application/x-votable+xml;content=datalink,https://data.lsst.cloud/api/datalink/links?ID=butler%3A//dp02/b876fbf2-9bbb-427d-8e86-e3c272054d34,3,lsst.deepCoadd_calexp,image,9.22e-07,8.18e-07,--,--,Rubin-LSST,,z,--,,17,4431,--,phot.count,LSST.DP02,DC2-4431-17,,--,-32.205663176664345,0.3299793994195083,55.65214108466622,POLYGON ICRS 55.514085 -32.322253 55.790197 -32.322253 55.789845 -32.088924 55.514437 -32.088924,--,--,--,--,--,--,--,--,
application/x-votable+xml;content=datalink,https://data.lsst.cloud/api/datalink/links?ID=butler%3A//dp02/f9ba9b77-3e1d-409c-b7f1-1be7e42ff790,3,lsst.deepCoadd_calexp,image,6.91e-07,5.52e-07,--,--,Rubin-LSST,,r,--,,17,4431,--,phot.count,LSST.DP02,DC2-4431-17,,--,-32.205663176664345,0.3299793994195083,55.65214108466622,POLYGON ICRS 55.514085 -32.322253 55.790197 -32.322253 55.789845 -32.088924 55.514437 -32.088924,--,--,--,--,--,--,--,--,
application/x-votable+xml;content=datalink,https://data.lsst.cloud/api/datalink/links?ID=butler%3A//dp02/3b7eb0b6-71cc-4b09-96dd-17293d57bf84,3,lsst.deepCoadd_calexp,image,5.52e-07,4.02e-07,--,--,Rubin-LSST,,g,--,,17,4431,--,phot.count,LSST.DP02,DC2-4431-17,,--,-32.205663176664345,0.3299793994195083,55.65214108466622,POLYGON ICRS 55.514085 -32.322253 55.790197 -32.322253 55.789845 -32.088924 55.514437 -32.088924,--,--,--,--,--,--,--,--,
application/x-votable+xml;content=datalink,https://data.lsst.cloud/api/datalink/links?ID=butler%3A//dp02/bcaafd06-fa5e-4b32-bf2e-351c23a4da83,3,lsst.deepCoadd_calexp,image,4e-07,3.3e-07,--,--,Rubin-LSST,,u,--,,17,4431,--,phot.count,LSST.DP02,DC2-4431-17,,--,-32.205663176664345,0.3299793994195083,55.65214108466622,POLYGON ICRS 55.514085 -32.322253 55.790197 -32.322253 55.789845 -32.088924 55.514437 -32.088924,--,--,--,--,--,--,--,--,
application/x-votable+xml;content=datalink,https://data.lsst.cloud/api/datalink/links?ID=butler%3A//dp02/20d28216-534a-4102-b8a7-1c7f32a9b78c,3,lsst.deepCoadd_calexp,image,8.18e-07,6.91e-07,--,--,Rubin-LSST,,i,--,,17,4431,--,phot.count,LSST.DP02,DC2-4431-17,,--,-32.205663176664345,0.3299793994195083,55.65214108466622,POLYGON ICRS 55.514085 -32.322253 55.790197 -32.322253 55.789845 -32.088924 55.514437 -32.088924,--,--,--,--,--,--,--,--,


We can query directly for the image above with the tract, patch, and filter 

In [22]:
query = """SELECT access_format, access_url, dataproduct_subtype, lsst_patch, lsst_tract, lsst_band, s_ra, s_dec
FROM ivoa.ObsCore 
WHERE dataproduct_type = 'image'
AND obs_collection = 'LSST.DP02' 
AND dataproduct_subtype = 'lsst.deepCoadd_calexp'
AND lsst_tract = 4431
AND lsst_patch = 17
AND lsst_band = 'i'
"""
results = service.search(query).to_table().show_in_notebook()
results

idx,access_format,access_url,dataproduct_subtype,lsst_patch,lsst_tract,lsst_band,s_ra,s_dec
,,,,,,,deg,deg
0,application/x-votable+xml;content=datalink,https://data.lsst.cloud/api/datalink/links?ID=butler%3A//dp02/20d28216-534a-4102-b8a7-1c7f32a9b78c,lsst.deepCoadd_calexp,17,4431,i,55.65214108466622,-32.205663176664345


The image UUID that we extracted from the Butler above is part of the access url field. We can query on that as well.  We passed a single UUID (in the first query) so there must be 1 result only (but there are zero)

In [ ]:
query = """
SELECT * FROM ivoa.ObsCore WHERE access_url like '%20d28216-534a-4102-b8a7-1c7f32a9b78c' 
"""
results = service.search(query).to_table()
assert len(results) == 1  
results

### 3.2 Accessing Image Data Products

Now that we know how to query the ObsCore table to find images, lets see how to access, manipulate and download them

In [ ]:
# Now lets retrieve the deep coadd from previously as well as one of the visit images that 
query = """SELECT access_format, access_url, dataproduct_subtype, lsst_patch, lsst_tract, lsst_band, s_ra, s_dec
FROM ivoa.ObsCore 
WHERE dataproduct_type = 'image'
AND obs_collection = 'LSST.DP02' 
AND dataproduct_subtype = 'lsst.deepCoadd_calexp'
AND lsst_tract = 4431
AND lsst_patch = 17
AND lsst_band = 'i'
"""
results = service.search(query)
results.to_table().show_in_notebook()

Take a look at the columns \"access_format\" and \access_url\". A access_url is provided for each image in a row from the ObsCore query.  <rsp-base-url>/api/datalink/links?ID=<id> where <rsp-base-url> is the base URL of the Rubin Science Platform and <id> is a UUID for that image in Butler. Recall previously that we were able to query by UUID. \"access_format\" tells us that the format is DataLinks. We can get the DataLinks URL as follows: 

In [ ]:
dataLinkUrl = results[0].getdataurl()
f"Datalink link service url: {dataLinkUrl}"

Before proceeding we need to extract the session authentiction for reuse. Explain more about this

In [ ]:
auth_session = service._session

We call call PyVo's DatalinkResults with the DataLink URL for the image we want to retrieve. This returns a list matching records, each record containoing a set of metadata describing the record.

In [ ]:
dl_results = DatalinkResults.from_result_url(dataLinkUrl,session=auth_session)
f"{dl_results.status}"

In [ ]:
dl_results.to_table().show_in_notebook()

In the table above we see that two records are returned for our image. The first is a signed URL for the image (Primary image or observation data file) that is valid for an hour. <ADD some text explaingin where the image is stored and the URL>. The second is a link to the cutout service. 

In [ ]:
image_url = dl_results.getrecord(0).get('access_url')
f"{image_url}"

We can plot the image by making an Exposure object directly from the URL. This plot looks identical to the plot of the same image retrived via the Butler

In [ ]:
new_coadd = ExposureF(image_url)
plotImage(new_coadd)

In [ ]:
# We can open the URL and take look at the headers 
hdulist = fits.open(image_url)
for hdu in hdulist:
    print(hdu.name)

In [ ]:
# We can also download the file
filename = download_file(image_url)
assert os.path.isfile(filename)
f"{filename}"

## 4 Image cutout service 

The Rubin Image Cutout Serivce is implemented using the [IVOA SODA](https://ivoa.net/documents/SODA/20170517/REC-SODA-1.0.html) standard. SODA (Server-side Operations for Data Access) is a low-level data access capability or server side data processing that can act upon data files, performing various kinds of operations, such as filtering/subsection, transformations, pixel operations, and applying functions to the data.

Need the Butler UUID of an image 

The initial implementation of the image cutout service will only return FITS files

Initial implementation supports 
CIRCLE and POLYGON.  POS=RANGE not implemented - check!

The initial version of the cutout service will only support a single ID parameter and a single stencil parameter.
The ID parameter must be a UUID assigned by the Butler and uniquely identifying a source image

The Image cutout service is based on SODA 1.0. 
Examples: 
https://github.com/astropy/pyvo/blob/main/examples/images/ex_get_cutouts.py
https://github.com/astropy/pyvo/blob/main/examples/images/ex_casA_image_cat.py

## 4.1 Using PyVO's SodaQuery object

Introduction to SODA

SODA, describe the acronym is a ..... , describe service
The Rubin SODA service is to support performing cutouts from the collected LSST image data.

Only synchronouse queries of the cutout service run currently. An asynchronous quert support may be introduced in the future

SODA needs the Butler UUID. Specifically, the URL is similar to https://data-int.lsst.cloud/api/cutout/sync?id=8a953c0321bd4878bfa694dbf628ea81&circle=53.13925%20-34.0215%200.0105 (the id parameter is the Butler UUID, and the remaining parameters are the cutout request, and there's a POST version as well and an async version following the SODA standard).

Now lets look at the second result in the DataLink table above. This record provides informaitn about the cutout service. We will use this to get a cutout by using SodaQuery 

Note:  add some description and an introduction here as to what SODA and a SODA service is. 

Prepare a query to the SODA service using the DataLink results from above and passing the session authorization token. 

In [ ]:
sq = SodaQuery.from_resource(dl_results, dl_results.get_adhocservice_by_id("cutout-sync"), 
                             session=auth_session)

Now define a a circle cutout centered on the galaxy cluster 

In [ ]:
sphereRadius = 0.03* u.deg
sq.circle = (spherePoint.getRa().asDegrees()* u.deg,
             spherePoint.getDec().asDegrees()*u.deg, 
             sphereRadius)
f"Circle around point {spherePoint} of radius {sphereRadius}"

In [ ]:
# Which shapes / stencils are supported currently
sq.

In [ ]:
#help(sq)

Now create the cutout 

In [ ]:
sodaCutout = os.path.join(os.getenv('HOME'), 'DATA/soda-cutout.fits')
with open(sodaCutout, 'bw') as f:
    f.write(sq.execute_stream().read())

In [ ]:
# Display the cutout
plotImage(ExposureF(sodaCutout))

We can see that the image cutout created by the cutout service looks identical to the one we obtained via the Butler

### 4.2 Using the SODA endpoint

We construct the SODA endoint as follows. This is similar to endpoints for TAP 

In [ ]:
host=os.getenv("EXTERNAL_INSTANCE_URL")
SODA_URL="{}/api/image/soda/sync".format(host)
ID='default.calexp.r'
print(SODA_URL)

## 5. Exercises to the user 

* Get the UUID for all the visits that comprise the deepCoadd, get a cutout of the galaxy cluster and display them in a grid of images 
* Do a reprocessing on a cutout
* Find an interesting object and use the cutout service to get cutouts of the object in all bands

## 6. References 

Rubin technotes
*  [DMTN-208](dmtn-208.lsst.io): RSP image cutout service implementation strategy
*  [DMTN-238](dmtn-238.lsst.io): RSP DataLink service implementation strategy
*  https://dmtn-139.lsst.io/v/DM-22746/index.html , section 7.4 (not published yet)


Relevant IVOA standards documents 
*  IVOA Table Access Protocol (TAP) https://www.ivoa.net/documents/TAP/